In [1]:
%reload_ext autoreload
%autoreload 1

In [2]:
import os
import sys
import warnings
import logging
warnings.filterwarnings('ignore')

curruser = os.environ.get('USER')

# sys.path.insert(0, '/opt/workspace/{user}/system/support_library/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/libs/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/system/labdata/lib/'.format(user=curruser))

# sys.path.insert(0, '/opt/workspace/{}/notebooks/Clickstream_Analytics/scr'.format(curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser)) 
sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata/lib/'.format(user=curruser))

# sys.path.insert(0, '/home/{}/notebooks/support_library/'.format(curruser)) 
# sys.path.insert(0, '/home/{}/python35-libs/lib/python3.5/site-packages/'.format(curruser))
# sys.path.insert(0, '/home/{}/notebooks/labdata/lib/'.format(curruser))

# from tqdm import tqdm
# from tqdm._tqdm_notebook import tqdm_notebook
# tqdm_notebook.pandas()

from pathlib import Path
import pandas as pd
import numpy as np
import re 
import joblib
import subprocess

# pd.set_option('display.float_format', lambda x: '%.3f' % x)
# pd.set_option('display.max_colwidth', -1)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

from spark_connector import SparkConnector
from sparkdb_loader import spark, log
# from spark_helper import SparkHelper
from connector import OracleDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext, SQLContext
from pyspark.sql.functions import udf 
from pyspark.sql import functions as f
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer

# import loader as load

# from corpora_process import utils
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [3]:
# src_path = os.getcwd() + '/../../src/'
# sys.path.insert(0, src_path) 

# from urlprocessing import ParseURL
# from collections import OrderedDict
# from CRMDictProds import *
from datetime import datetime

#----------------------------------------------------------------------------#
#----------------------------------------------------------------------------#

# logging.basicConfig(filename='./__upd_NRT_CLICKSTREAM__.log',
#                     level=logging.INFO,
#                     format='%(asctime)s %(levelname)s %(name)s %(message)s')
# logger = logging.getLogger(__name__)

#----------------------------------------------------------------------------#
#----------------------------------------------------------------------------#

In [4]:
def show(self, n=10):
    return self.limit(n).toPandas()

pyspark.sql.dataframe.DataFrame.show = show

In [5]:
os.environ['SPARK_CLASSPATH']

'/opt/cloudera/parcels/PYENV.ZNO0059623792/usr/lib/oracle/12.2/client64/lib/ojdbc8.jar:/home/ektov1-av_ca-sbrf-ru/notebooks/drivers/sqoop-connector-teradata-1.7c5.jar:/home/ektov1-av_ca-sbrf-ru/notebooks/drivers/tdgssconfig.jar:/home/ektov1-av_ca-sbrf-ru/notebooks/drivers/terajdbc4.jar'

In [8]:
%%time
sp = spark(schema='sklod_external_google_analytics',
           queue = 'ektov1-av_ca-sbrf-ru',
           process_label="NRT_SBBOL_",
           numofinstances=8, 
           numofcores=8,
           executor_memory='25g', 
           driver_memory='25g',
           dynamic_alloc=False,
           kerberos_auth=True,
           need_oracle=True
          ) 
print(sp.sc.version)
hive = sp.sql

2.4.0.cloudera2
CPU times: user 32.5 ms, sys: 10.9 ms, total: 43.4 ms
Wall time: 15.3 s


====================================================== 2022-04-17 ======================================================
# __init__ : begin


In [7]:
sp.sc.stop()

In [9]:
dict(sp.sc.getConf().getAll())

{'spark.app.id': 'application_1645879035230_1950',
 'spark.app.name': 'NRT_SBBOL__sklod_external_google_analytics',
 'spark.authenticate': 'false',
 'spark.blacklist.enabled': 'true',
 'spark.blacklist.task.maxTaskAttemptsPerExecutor': '13',
 'spark.blacklist.task.maxTaskAttemptsPerNode': '13',
 'spark.default.parallelism': '80',
 'spark.driver.allowMultipleContexts': 'true',
 'spark.driver.appUIAddress': 'http://pklis-chd002207.labiac.df.sbrf.ru:4448',
 'spark.driver.extraClassPath': '/opt/cloudera/parcels/PYENV.ZNO0059623792/usr/lib/oracle/12.2/client64/lib/ojdbc8.jar:/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/drivers/hive-jdbc-1.1.0-cdh5.16.2-standalone.jar:/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/drivers/hive-exec-1.1.0-cdh5.16.2.jar:/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/drivers/hive-service-1.1.0-cdh5.16.2.jar:/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/drivers/hive-metastore-1.1.0-cdh5.16.2.jar:/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/drivers/commons-config

## Use `SparkSession` as spark runner

In [21]:
conf_get = sp.sc._conf.getAll()

In [22]:
conf = SparkConf().setAll(conf_get)

In [70]:
# sp.sc.stop()

In [6]:
spark = SparkSession.builder.config('spark.master', 'yarn')\
                            .config('spark.hadoop.hive.metastore.uris', 'thrift://pklis-chd002193.labiac.df.sbrf.ru:48869')\
                            .config('spark.driver.maxResultSize', '90g')\
                            .config('spark.driver.memory', '30g')\
                            .config('spark.executor.cores', '8')\
                            .config('spark.executor.instances', '8')\
                            .config('spark.executor.memory', '25g')\
                            .config('spark.yarn.driver.memoryOverhead', '4048mb')\
                            .config('spark.yarn.executor.memoryOverhead', '8g')\
                            .config('spark.kryoserializer.buffer.max', '1024mb')\
                            .config('spark.dynamicAllocation.enabled', 'false')\
                            .config('spark.ui.port', '4445')\
                            .config('spark.yarn.access.namenodes', 'hdfs://nsld3:8020/,hdfs://clsklsbx:8020/')\
                            .config('spark.sql.hive.metastore.jars', '${env:HADOOP_COMMON_HOME}/../hive/lib/*:${env:HADOOP_COMMON_HOME}/client/*')\
                            .config('spark.yarn.jars', 'local:/opt/cloudera/parcels/SPARK2-2.4.0.cloudera2-1.cdh5.13.3.p0.1041012/lib/spark2/jars/*')\
                            .config('spark.repl.local.jars', 'file:///opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/drivers/hive-apache-jdbc-0.13.1-9.jar')\
                            .config('spark.yarn.dist.jars', '/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/drivers/hive-loaderfactory-02.14.00.00-1.jar, /opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/drivers/hive-apache-jdbc-0.13.1-9.jar')\
                            .enableHiveSupport().getOrCreate()
                            
sc= spark.sparkContext
hive = HiveContext(sc)                            

In [13]:
sc._conf.getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.yarn.driver.memoryOverhead', '4048mb'),
 ('spark.sql.queryExecutionListeners',
  'com.cloudera.spark.lineage.NavigatorQueryListener'),
 ('spark.eventLog.dir', 'hdfs://nsld3/user/spark/spark2ApplicationHistory'),
 ('spark.ui.port', '4445'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.sql.hive.metastore.jars',
  '${env:HADOOP_COMMON_HOME}/../hive/lib/*:${env:HADOOP_COMMON_HOME}/client/*'),
 ('spark.lineage.log.dir', '/var/log/spark2/lineage'),
 ('spark.driver.port', '44115'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 ('spark.driver.memory', '30g'),
 ('spark.driver.extraLibraryPath',
  '/opt/cloudera/parcels/CDH-5.16.2-1.cdh5.16.2.p0.8/lib/hadoop/lib/native'),
 ('spark.network.crypto.enabled', 'false'),
 ('spark.driver.host', 'pklis-chd002207.labiac.df.sbrf.ru'),
 ('spark.ui.enabled', 'true'),
 ('spark.executor.id', 'driver'),
 ('spark.dynamicAllocation.schedulerBa

## `ADD JARS` via HiveContext

In [8]:
hive.sql("show databases").show()

,databaseName
0,default
1,td15_prd_db_client4d_cvm
2,td15_prd_db_client4d_dev1
3,td15_prd_db_client4d_mis
4,td15_sbx_db_sme_dkk_analyse
5,sbx_core_internal_client4d_mis
6,sbx_core_internal_client4d_seldon_zakupki
7,sbx_core_internal_db_dmp4dplg
8,sbx_core_internal_integrum
9,sbx_core_internal_pravo


## Run Queries Throught HiveContext

## Hive-TDQG connector

In [9]:
hive.sql("list jars").collect()

[Row(Results='spark://pklis-chd002207.labiac.df.sbrf.ru:43388/jars/hive-exec-1.1.0-cdh5.16.2.jar'),
 Row(Results='spark://pklis-chd002207.labiac.df.sbrf.ru:43388/jars/hive-metastore-1.1.0-cdh5.16.2.jar'),
 Row(Results='spark://pklis-chd002207.labiac.df.sbrf.ru:43388/jars/commons-configuration-1.6.jar'),
 Row(Results='spark://pklis-chd002207.labiac.df.sbrf.ru:43388/jars/hive-jdbc-1.1.0-cdh5.16.2-standalone.jar'),
 Row(Results='spark://pklis-chd002207.labiac.df.sbrf.ru:43388/jars/libthrift-0.9.0.jar'),
 Row(Results='spark://pklis-chd002207.labiac.df.sbrf.ru:43388/jars/hadoop-common-2.6.0-cdh5.16.2.jar'),
 Row(Results='spark://pklis-chd002207.labiac.df.sbrf.ru:43388/jars/terajdbc4.jar'),
 Row(Results='spark://pklis-chd002207.labiac.df.sbrf.ru:43388/jars/commons-logging-1.1.3.jar'),
 Row(Results='spark://pklis-chd002207.labiac.df.sbrf.ru:43388/jars/httpcore-4.2.5.jar'),
 Row(Results='spark://pklis-chd002207.labiac.df.sbrf.ru:43388/jars/tdgssconfig.jar'),
 Row(Results='spark://pklis-chd0022

In [12]:
DRIVERS_PATH = "/opt/workspace/{}/notebooks/drivers/".format(curruser)
DRIVERS_ALL = "/opt/workspace/{}/notebooks/drivers/*".format(curruser)
AUX_JAR_HDFS_PATH = "hdfs:///user/{}/tdqg/jars/".format(curruser)
AUX_JARS = "hdfs:///user/{}/tdqg/jars/*".format(curruser)
JAR_PATH = "/opt/workspace/{}/notebooks/drivers/hive-loaderfactory-02.14.00.00-1.jar".format(curruser)

In [23]:
! hdfs dfs -mkdir -p hdfs:///user/ektov1-av_ca-sbrf-ru/tdqg/jars/

In [15]:
!hdfs dfs -rm -f -skipTrash hdfs:///user/ektov1-av_ca-sbrf-ru/tdqg/jars/*

In [16]:
! hdfs dfs -put -f /opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/drivers/hive-loaderfactory* hdfs:///user/ektov1-av_ca-sbrf-ru/tdqg/jars/

In [13]:
# hive.setConf("hive.aux.jars.path", AUX_JARS)

hive.sql("ADD JAR {}".format(JAR_PATH))

DataFrame[result: int]

In [14]:
sdf = hive.table("td15_prd_db_client4d_dev1.ga_all_scenarios_hist")

In [ ]:
sdf.show()

## HiveJdbc

In [23]:
!PASS=$(cat /home/$(whoami)/pass/userpswrd | sed 's/\r//g') && printf "$PASS" | kinit $(whoami)@DF.SBRF.RU

Password for ektov1-av_ca-sbrf-ru@DF.SBRF.RU: 


In [10]:
BATCH_SIZE = 100000

uri="jdbc:hive2://pklis-chd002185.labiac.df.sbrf.ru:10000/sbx_t_team_cvm;principal=hive/_HOST@$DF.SBRF.RU"

sql_query="(select * from sbx_t_team_cvm.lal_db_hist_out where create_dt_day='2022-03-24') as t"

sdf=\
hive\
    .read \
    .format('jdbc') \
    .option('url', uri) \
    .option('dbtable', sql_query) \
    .option('driver', 'org.apache.hive.jdbc.HiveDriver') \
    .option('fetchsize', BATCH_SIZE) \
    .load()

In [ ]:
sdf.take(10)